# Course Name :- Stochastic Processes [ MTH332 ]
# Course Instructor :- Dr. LokPati Tripathi
## Author Name :
          1. Sarthak [ 1906332 ]
          2. Nishant Kumar [ 1904123 ]

#Assignment-3B: Monte Carlo Integration - A machine learning approach #

Apply the algorithm proposed in the report "A machine learning approach for efficient multi‐dimensional
integration" to approximate the value of  

1.   single-asset European style call option. 

    Pay-off function: $V(S,T) = \max(S-K, 0)$ 
    
    Parameters: $T = 1, r=0.05, K=100, S_{0}=110, \text{ and } \sigma = 0.2$
2.   single-asset European style put option. 

    Pay-off function: $V(S,T) = \max(K-S, 0)$

    Parameters: $T = 1, r=0.05, K=100, S_{0}=90, \text{ and } \sigma = 0.2$
3.   multi-asset European style rainbow put on min option. It gives the holder the right to sell the minimum asset at the strike price K at expiry T. 

  Pay-off function: $V(S^{1}, S^{2},\ldots S^{n}, T) = \max\{K - \min(S^{1}, S^{2},\ldots,S^{n}),0\}$

  Parameters: $T = 1, r=0.05, K=100, S^{i}_{0}=100,\; \rho_{ii} = 1,\;\rho_{ij} = 0.1, i\neq j, \text{ and } \sigma_{i} = 0.2$

  Compute the value of the rainbow option for two-asset(n=2) and four-asset(n=4) and compare the result with the exact solution in two-asset case.

Describe the results which you obtain after applying the algorithm.

*Note:* Please explain clearly all the parameters, specific terms, and notations involved in your solution.








In [ ]:
!pip install vegas

     |████████████████████████████████| 1.3 MB 5.1 MB/s 
     |████████████████████████████████| 1.9 MB 29.1 MB/s 
  Created wheel for vegas: filename=vegas-5.1.1-cp37-cp37m-linux_x86_64.whl size=1501875 sha256=7278b554d2f8488c5b6e904a76b80e8ed833800271edf0b63a85bb88da4c0c6c
  Stored in directory: /root/.cache/pip/wheels/5c/e4/05/3bcde90bc37d2c2b77a85b376b36926daee5516139fa9cd610
  Created wheel for gvar: filename=gvar-11.9.6-cp37-cp37m-linux_x86_64.whl size=3947973 sha256=df8f9549d9a8d86d7fd14c4a9a5ce0f250179c602f09dac9ab5f25f48f8ba7a6
  Stored in directory: /root/.cache/pip/wheels/2b/fd/bf/40b85a16253565a4f9414bceb6278ea7bebf7ea5c9b6ae9be6
Successfully built vegas gvar


# A 

$$ Single-asset * European * style * call * option.$$

$$ Pay-off *function: V(S,T) = \max(S-K, 0) $$
    
$$ Parameters: T = 1, r=0.05, K=100, S_{0}=110, \text{ and } \sigma = 0.2 $$

In [414]:
# Importing required libraries
import numpy as np

# number of samples
n = 10000

# Sampling normal distribution
X = np.random.normal(0, 1, n)

### Formula of S(t) :-
$$ \Large S(t) = S0 * e^{(\mu - \frac{1}{2}\sigma^2)t + \sigma\sqrt(t)Y} $$

$$ \Large Y \sim \mathcal{N}(0, 1)$$

In [415]:
# Parameter for European style call option
# maturity 
T = 1

 # risk free rate
r = 0.05

# strike price
K = 100

# spot stock price
S0 = 110

# diffusion coefficient or volatility
sig = 0.2

In [426]:
# Defining St
def PriceAtT1(X):
    return S0 * np.exp((r - 1 / 2 * sig ** 2) * T + sig * np.sqrt(T) * X)

$$ PayOff = MAX(St - K, 0)$$

In [427]:
# Pay Off Function of European Call Option
Y = np.maximum(PriceAtT1(X) - K, 0)

### Fitting Values to Multi-layer Perceptron Model

In [428]:
# Defining MLPClassifier
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver = 'lbfgs', alpha = 1e-5, hidden_layer_sizes = (4, 3), random_state = 1)

# Reshaping input to 2d
X = np.reshape(X, (len(X), 1))
Y = Y.astype('int')

# Fitting X, Y values.
clf.fit(X, Y)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(4, 3), random_state=1,
              solver='lbfgs')

In [432]:
Y = clf.predict(X)
print("Direct Prediction of Pay off using MLP classifier :- ", np.mean(Y))

Direct Prediction of Pay off using MLP classifier :-  17.377


### Monte Carlo Simulation Results

In [381]:
# Simulation using Monte Carlo method
def Monte_Carlo_Simulation1():
    Total_Sum = 0
    for i in range(1000):
        X = np.random.normal(0, 1, 10000)
        Y = np.maximum(PriceAtT1(X) - K, 0)
        Total_Sum += np.mean(Y)
    return Total_Sum / 1000
  
result = Monte_Carlo_Simulation1()
result * np.exp(-r * T)

17.665784155258333

### Using Vegas 

In [85]:
def f1(X):
    return max(S0 * np.exp((r - 1 / 2 * (sig ** 2)) * T + sig * np.sqrt(T) * X) - K, 0)

In [86]:
# importing required libraries
import vegas
import math
import gvar as gv

# integrator for expectation values in distribution g
g =  gv.gvar(0, 1)
g_expval = vegas.PDFIntegrator(g)

# <f1> in distribution g
results = g_expval(f1, nitn = 5, neval = 10000)
print(results * np.exp(- r * T))

17.6620(16)


### Using Scipy integrator

In [87]:
def f2(X):
    return max(S0 * np.exp((r - 1/2 * sig ** 2) * T + sig * np.sqrt(T) * X) - K, 0) * (1 / np.sqrt(2 * np.pi)) * np.exp(- (1 / 2) * (X ** 2))

In [88]:
from scipy.integrate import quad
def integrate1():
    return quad(f2, -np.inf, np.inf)

In [89]:
Integral = integrate1()
Integral[0] * np.exp(-r * T)

17.662953759487184

## The price P of the option at is given by P0 := P(S0, 0) of european style call option for single asset :-
1. using Monte Carlo Simulation is : 17.861992467235645
2. using Vegas Integrator is : 17.6620(16)
3. using Scipy Integrator is : 17.662953759487184

#######################################################################################################

# B

$$ Single - asset* European *style*put *option. $$

$$ Pay-off*function: V(S,T) = \max(K-S, 0) $$

$$ Parameters: T = 1, r=0.05, K=100, S_{0}=90, \text{ and } \sigma = 0.2$$

In [288]:
# Importing required libraries
import numpy as np

# number of samples
n = 10000

# Sampling normal distribution
X = np.random.normal(0, 1, n)

In [289]:
# Parameter for European style call option

# maturity 
T = 1

# risk free rate
r = 0.05

# strike price
K = 100

# stock price
S0 = 90

# diffusion coefficient volatility
sig = 0.2

In [290]:
# Defining St
def PriceAtT2(X):
    return S0 * np.exp((r - 1 / 2 * sig ** 2) * T + sig * np.sqrt(T) * X)

$$ PayOff = MAX(K - St, 0)$$

In [304]:
# Price of European Put Option at T = 0
Y = np.maximum(K - PriceAtT2(X), 0) * np.exp(- r * T)

### Monte Carlo Simulation Results

In [305]:
def Monte_Carlo_Simulation2():
    Total_Sum = 0
    for i in range(1000):
        X = np.random.normal(0, 1, 10000)
        Y = np.maximum(K - PriceAtT2(X), 0) * np.exp(- r * T)
        Total_Sum += np.mean(Y)
    return Total_Sum / 1000
  
result = Monte_Carlo_Simulation2()
result

10.206868326622919

### Using Vegas 

In [306]:
def f3(X):
    return max(K - S0 * np.exp((r - (1 / 2) * (sig ** 2)) * T + sig * np.sqrt(T) * X), 0)

In [307]:
# improting Vegas Libraries
import vegas
import math
import gvar as gv

# integrator for expectation values in distribution g
g =  gv.gvar(0, 1)
g_expval = vegas.PDFIntegrator(g)

# <f3> in distribution g
results = g_expval(f3, nitn = 5, neval = 10000)
print(results * np.exp(- r * T))

10.21416(64)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


### Using Scipy Integrator

In [308]:
def f4(X):
    return max(K - S0 * np.exp((r - (1 / 2) * (sig ** 2)) * T + sig * np.sqrt(T) * X), 0) * (1 / np.sqrt(2 * np.pi)) * np.exp(- (1 / 2) * (X ** 2))

In [309]:
from scipy.integrate import quad
def integrate():
    return quad(f4, -np.inf, np.inf)

Integral = integrate()
Integral[0] * np.exp(- r * T)

10.214164537066113

## The price P of the option at is given by P0 := P(S0, 0) of european style put option for single asset :- 
1. using Monte Carlo Simulation is : 10.206868326622919
2. using Vegas Integrator is : 10.21416(64)
3. using Scipy Integrator is : 10.214164537066113

#################################################################################################

# C

Multi-asset European style rainbow put on min option. It gives the holder the right to sell the minimum asset at the strike price K at expiry T. 

  Pay-off function: $V(S^{1}, S^{2},\ldots S^{n}, T) = \max\{K - \min(S^{1}, S^{2},\ldots,S^{n}),0\}$

  Parameters: $T = 1, r=0.05, K=100, S^{i}_{0}=100,\; \rho_{ii} = 1,\;\rho_{ij} = 0.1, i\neq j, \text{ and } \sigma_{i} = 0.2$

  Compute the value of the rainbow option for two-asset(n=2) and four-asset(n=4) and compare the result with the exact solution in two-asset case.

### Two Asset European Style Put Option

In [354]:
# Importing required libraries
import numpy as np

# number of samples
n = 100000

# Sampling 1st normal distribution
X = np.random.normal(0, 1, n)

In [355]:
# maturity data
T = 1

# risk free rate
r = 0.05

# strike price
K = 100

# stock price
S = [100, 100]

# volatility
sig = 0.2

In [356]:
# Defining St
def PriceAtT3(S, X):
    return S * np.exp((r - 1 / 2 * sig ** 2) * T + sig * np.sqrt(T) * X)

### Monte Carlo Simulation Results

In [359]:
# Monte Carlo results
def Monte_Carlo_Simulation3():
    Total_Sum = 0
    for i in range(1000):
        X1 = np.random.normal(0, 1, 10000)
        X2 = np.random.normal(0, 1, 10000)
        Y = np.maximum(K - np.minimum(PriceAtT3(S[0], X1), PriceAtT3(S[1], X2)), 0)
        Total_Sum += np.mean(Y)
    return Total_Sum / 1000
  
result = Monte_Carlo_Simulation3()
result

10.164907277019806

### Using Vegas Integrator

In [360]:
# Covariance Matrix
Sig = np.array([0.20, 0.20])

# Mean 
mean = [0, 0]

# Correlation Matrix
Rho = np.array([[1, 0.1], [0.1, 1]])

In [361]:
def f5(X):
  [a, b] = S * np.exp((r - (1 / 2) * (Sig ** 2)) * T + Sig * np.sqrt(T) * X)
  q = min(a, b)
  return max(K - q, 0)

In [362]:
# improting Vegas Libraries
import vegas
import math
import gvar as gv

# integrator for expectation values in distribution g
g =  gv.gvar(mean, Rho)
g_expval = vegas.PDFIntegrator(g)

# <f5> in distribution g
results = g_expval(f5, nitn = 5, neval = 10000)
print(results)

9.9394(77)


## The price P of the option at is given by P0 := P(S0, 0) of european style put option for two asset :-
1. Using Monte Carlo :- 10.164907277019806
2. Using Vegas Integrator :- 9.9394(77)

### Four Asset European Style Put Option

In [363]:
# Covariance Matrix
Sig_2 = np.array([0.20, 0.20, 0.20, 0.20])

# Mean Matrix
mean = np.array([0, 0, 0, 0])

# stock price 
S = [100, 100, 100, 100]

# risk free rate
r = 0.05

# Correlation Matrix
Rho_2 = np.array([[1, 0.1, 0.1, 0.1], [0.1, 1, 0.1, 0.1], [0.1, 0.1, 1, 0.1], [0.1, 0.1, 0.1, 1]])

In [364]:
def f(X):
  [a, b, c, d] = S * np.exp((r - (1 / 2) * (Sig_2 ** 2)) * T + Sig_2 * np.sqrt(T) * X)
  q = min(min(min(a, b), c), d)
  return max(K - q, 0)

In [365]:
# importing Vegas Libraries
import vegas
import math
import gvar as gv

# integrator for expectation values in distribution g
g = gv.gvar(mean, Rho_2)
g_expval = vegas.PDFIntegrator(g)

# <f> in distribution g
results = g_expval(f, nitn = 5, neval = 10000)
print(results* np.exp(- r * T))

14.57(13)


## The price P of the option at is given by P0 := P(S0, 0) of european style put option for four asset :-
1. Using Vegas Integrator : 14.57(13)